In [1]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os

In [3]:
#Define path
SECRETS_FILE = '/Users/gmccurdy/repos/curriculum/scripts/utilities/secrets.json'

# Load secrets
with open(SECRETS_FILE, 'r') as file:
    secrets = json.load(file)

# Assign variables from the secrets
API_URL = secrets.get('API_URL')
API_KEY = secrets.get('API_KEY')
COURSE_ID = secrets.get('COURSE_ID_APSTATISTICS')

#Define headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [4]:
# Create the quiz
quiz_title = input('Quiz Title')
quiz_description = input('Quiz Description')
quiz_data = {
    'quiz': {
        'title': quiz_title,
        'description': f'<p>{quiz_description}</p>',
        'quiz_type': 'assignment',
        'published': False
    }
}

create_quiz_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes'
response = requests.post(create_quiz_url, headers=headers, json=quiz_data)

if response.status_code == 200:
    quiz = response.json()
    quiz_id = quiz['id']
    print(f'Quiz created successfully with ID: {quiz_id}')
else:
    print('Failed to create quiz')
    print(f"Status Code: {response.status_code}")
    print(response.text)

Quiz created successfully with ID: 44035


In [5]:
multiple_choice_questions = [
    {
        'question_name': 'Conditions for CLT for Sample Proportion',
        'question_type': 'multiple_choice_question',
        'question_text': '''Which of the following conditions must be satisfied to apply the Central Limit Theorem (CLT) for the sampling distribution of a sample proportion?''',
        'points_possible': 1,
        'answers': [
            {'text': 'The sample size must be at least 30', 'weight': 0},
            {'text': 'np ≥ 10 and n(1-p) ≥ 10', 'weight': 100},  # Correct answer
            {'text': 'The population must be normally distributed', 'weight': 0},
            {'text': 'The sample must be random', 'weight': 0},
            {'text': 'p must be greater than 0.5', 'weight': 0}
        ]
    },
    {
        'question_name': 'Standard Error of Sample Mean',
        'question_type': 'multiple_choice_question',
        'question_text': '''In a population with a mean of 50 and a standard deviation of 5, what is the **standard error** of the sampling distribution of the sample mean for a sample size of 25?''',
        'points_possible': 1,
        'answers': [
            {'text': '1.00', 'weight': 100},  # Correct answer: SE = 5 / sqrt(25) = 1.00
            {'text': '0.20', 'weight': 0},
            {'text': '5.00', 'weight': 0},
            {'text': '0.40', 'weight': 0},
            {'text': '0.50', 'weight': 0}
        ]
    },
    {
        'question_name': 'Probability of Sample Mean Greater Than a Value',
        'question_type': 'multiple_choice_question',
        'question_text': '''In a population with a mean of 100 and a standard deviation of 20, a sample of size 25 is taken. What is the **probability** that the sample mean is greater than 108? (Use the z-score table.)''',
        'points_possible': 1,
        'answers': [
            {'text': '0.1587', 'weight': 0},
            {'text': '0.0668', 'weight': 0},
            {'text': '0.8413', 'weight': 0},
            {'text': '0.3085', 'weight': 0},
            {'text': '0.0228', 'weight': 100}  # Correct answer: z = (108-100)/(20/sqrt(25)) = 8/4 = 2.00, P(z > 2) = 0.0228
        ]
    }
]

In [6]:
essay_questions = [

]

In [7]:
numerical_questions = [
    {
        'question_name': 'Z-Score for Sample Mean',
        'question_type': 'numerical_question',
        'question_text': '''In a population with a mean of 80 and a standard deviation of 12, a sample of size 36 is taken. What is the **z-score** for a sample mean of 86? Round your answer to two decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '3.00'  # Correct answer: z = (86 - 80) / (12 / sqrt(36)) = 6 / 2 = 3.00
            }
        ]
    },
    {
        'question_name': 'Standard Error of Sample Proportion',
        'question_type': 'numerical_question',
        'question_text': '''In a population with a proportion p = 0.3, a sample of size 100 is taken. What is the **standard error** of the sampling distribution of the sample proportion? Round your answer to four decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '0.0458'  # Correct answer: SE = sqrt(p(1-p)/n) = sqrt(0.3*0.7/100) = 0.0458
            }
        ]
    }
]

In [8]:
# Combine all questions, including the essay question
questions = multiple_choice_questions + numerical_questions + essay_questions

# Add questions to the quiz
for question in questions:
    create_question_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes/{quiz_id}/questions'

    # Base question data
    question_data = {
        'question': {
            'question_name': question['question_name'],
            'question_type': question['question_type'],
            'question_text': question['question_text'],
            'points_possible': question.get('points_possible', 1)
            # Do not include 'answers' here
        }
    }

    # Include 'answers' only if they exist and the question is not an essay question
    if 'answers' in question and question['question_type'] != 'essay_question':
        question_data['question']['answers'] = question['answers']

    # Include 'general_feedback' if it exists (for essay questions)
    if 'general_feedback' in question:
        question_data['question']['general_feedback'] = question['general_feedback']

    # Send the request to create the question
    response = requests.post(create_question_url, headers=headers, json=question_data)

    if response.status_code == 201 or response.status_code == 200:
        print(f"Added question '{question['question_name']}' successfully.")
    else:
        print(f"Failed to add question '{question['question_name']}'")
        print(f"Status Code: {response.status_code}")
        print(response.text)

Added question 'Conditions for CLT for Sample Proportion' successfully.
Added question 'Standard Error of Sample Mean' successfully.
Added question 'Probability of Sample Mean Greater Than a Value' successfully.
Added question 'Z-Score for Sample Mean' successfully.
Added question 'Standard Error of Sample Proportion' successfully.
